In [93]:
from login import login

import requests
import pandas as pd
import json
import api
import rh_positions as positions
import sys

rh = login()

def get_filled_orders(limit=sys.maxsize):
    orders = api.get_all_history_orders(rh)[:limit]
    orders = [api.order_item_info(order, rh) for order in orders]
    df = pd.DataFrame.from_records(orders)
    filled_orders = df.loc[df['state'] == 'filled']
    filled_orders = filled_orders.sort_values('date')
    return filled_orders

def get_realized_profits(filled_orders):
    positions = {}
    profits = {}
    for _, row in filled_orders.iterrows():
        tick, side = row[['symbol', 'side']]
        
        quantity, price = int(float(row['shares'])), float(row['price'])
        profits.setdefault(tick, 0)
        positions.setdefault(tick, [])

        if side == 'buy':
            positions[tick] += [price] * quantity
        else:
            spent = 0
            for _ in range(quantity):
                if positions[tick]:
                    spent += positions[tick].pop()
                else: break
            profits[tick] += price * quantity - spent

    return profits

def get_unrealized_profits(positions_df):
    profits = {}
    
    for _, row in positions_df.iterrows():
        quantity = int(float(row['quantity']))
        if quantity > 0:
            tick = row['instrument']
            price = float(row['current_price'])
            buy_price = float(row['average_buy_price'])
            # print(f'ticker: {tick} price: {price} buy price: {buy_price}')
            profits.setdefault(tick, 0)
            profits[tick] = (price - buy_price) * quantity 
    return profits

def get_holdings(positions):
    holdings = {}
    equity = {}

    for ticker in positions:
        holdings[ticker] = len(positions[ticker])
        equity[ticker] = sum(positions[ticker])
    
    return holdings, equity

def get_stats(filled_orders, positions_df):
    real = get_realized_profits(filled_orders)
    unreal = get_unrealized_profits(positions_df)

    return real, unreal

if __name__ == '__main__':
    orders = get_filled_orders()
    positions_df = positions.positions_to_df()

    stats = get_stats(orders, positions_df)

    profits = pd.DataFrame.from_records(stats, index=['Realized', 'Unrealized']).transpose().fillna(0)
    profits.index.name = 'instrument'
    profits = positions_df[['instrument', 'quantity', 'average_buy_price']].merge(profits, left_on='instrument', right_index=True)
    profits['average_buy_price'] = profits['average_buy_price'].astype(float) * profits['quantity'].astype(float)
    profits = profits.rename(columns={'average_buy_price' : 'equity'})
    profits['Total Profit'] = profits['Realized'] + profits['Unrealized']
    print(profits)



100 order fetched
116 order fetched
   instrument     quantity     equity   Realized  Unrealized  Total Profit
0        ILMN   4.00000000  1115.3800     0.0000    165.8600      165.8600
1       RHHBY  25.00000000  1009.7500     0.0000    104.7500      104.7500
2        BABA   5.00000000  1104.9600     0.0000    -77.9600      -77.9600
3        MSFT   5.00000000   841.2300     0.0000     92.4700       92.4700
4        AAPL   2.00000000   581.8600     0.0000     48.1600       48.1600
5          MA   2.00000000   552.0000     0.0000     10.4600       10.4600
6         DIS   5.00000000   629.0000     0.0000    -90.1500      -90.1500
7        AMAT  10.00000000   512.4200     0.0000     27.6800       27.6800
8           T  16.00000000   485.2800     0.0000    -15.2000      -15.2000
9         NEE   2.00000000   440.2900     0.0000     14.4700       14.4700
10        IYW   2.00000000   430.4200     0.0000     60.7400       60.7400
11       MRNA   7.00000000   219.2680   194.9701    249.0320    

In [94]:
profits

,instrument,quantity,equity,Realized,Unrealized,Total Profit
0,ILMN,4.00000000,1115.3800,0.0000,165.8600,165.8600
1,RHHBY,25.00000000,1009.7500,0.0000,104.7500,104.7500
2,BABA,5.00000000,1104.9600,0.0000,-77.9600,-77.9600
3,MSFT,5.00000000,841.2300,0.0000,92.4700,92.4700
4,AAPL,2.00000000,581.8600,0.0000,48.1600,48.1600
5,MA,2.00000000,552.0000,0.0000,10.4600,10.4600
6,DIS,5.00000000,629.0000,0.0000,-90.1500,-90.1500
7,AMAT,10.00000000,512.4200,0.0000,27.6800,27.6800
8,T,16.00000000,485.2800,0.0000,-15.2000,-15.2000
9,NEE,2.00000000,440.2900,0.0000,14.4700,14.4700
